# <font color = blue> IMDb Movie Analysis </font>

We have the data for the 100 top-rated movies from the past decade along with various pieces of information about the movie, its actors, and the voters who have rated these movies online. In this analysis, we will try to find some interesting insights into these movies and their voters, using Python.

In [ ]:
# Filtering out the warnings

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importing the required libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

##  Task 1: Reading the data

- ### Subtask 1.1: Read the Movies Data.

Read the movies data file provided and store it in a dataframe `movies`.

In [ ]:
# Read the csv file using 'read_csv'. Please write your dataset location here.
movies = pd.read_csv("../input/imdb-top-100-movies-in-the-past-decade/MovieAssignmentData.csv")

- ###  Subtask 1.2: Inspect the Dataframe

Inspect the dataframe for dimensions, null-values, and summary of different numeric columns.

In [ ]:
# Check the number of rows and columns in the dataframe
movies.shape

In [ ]:
# Check the column-wise info of the dataframe
movies.info()

In [ ]:
# Check the summary for the numeric columns 
movies.describe()

## Task 2: Data Analysis

Now that we have loaded the dataset and inspected it, we see that most of the data is in place. As of now, no data cleaning is required, so let's start with some data manipulation, analysis, and visualisation to get various insights about the data. 

-  ###  Subtask 2.1: Reduce those Digits!

These numbers in the `budget` and `gross` are too big, compromising its readability. Let's convert the unit of the `budget` and `gross` columns from `$` to `million $` first.

In [ ]:
# Inspecting gross and budget columns
movies.loc[:,["Gross","budget"]].head()

In [ ]:
# Divide the 'gross' and 'budget' columns by 1000000 to convert '$' to 'million $'
movies["Gross"] = movies["Gross"] / 1000000
movies["budget"] = movies["budget"] / 1000000

In [ ]:
# Checking gross and budget column values post change
movies.loc[:,["Gross","budget"]].head()

-  ###  Subtask 2.2: Let's Talk Profit!

    1. Create a new column called `profit` which contains the difference of the two columns: `gross` and `budget`.
    2. Sort the dataframe using the `profit` column as reference.
    3. Extract the top ten profiting movies in descending order and store them in a new dataframe - `top10`.
    4. Plot a scatter or a joint plot between the columns `budget` and `profit` and write a few words on what you observed.
    5. Extract the movies with a negative profit and store them in a new dataframe - `neg_profit`

In [ ]:
# Create the new column named 'profit' by subtracting the 'budget' column from the 'gross' column
# Create a new column called profit which contains the difference of the two columns: gross and budget

movies["profit"] = movies["Gross"] - movies["budget"]

In [ ]:
# Inspecting the data frame to check if "profit" column exists
movies.head(1)

In [ ]:
# Sort the dataframe with the 'profit' column as reference using the 'sort_values' function. Make sure to set the argument
#'ascending' to 'False'
movies.sort_values(by = "profit", ascending = False, inplace = True)

In [ ]:
# Resetting index after sorting by profit to start ordering at 0
movies.reset_index(drop = True, inplace = True)

In [ ]:
# Check to see if profit is in descending order
movies.head(3)

In [ ]:
# Get the top 10 profitable movies by using position based indexing. Specify the rows till 10 (0-9)
# Extract the top ten profiting movies in descending order and store them in a new dataframe - top10
top10 = movies.loc[:9,:].copy()

In [ ]:
# Checking if we have top 10 movies in the dataframe
top10.shape

In [ ]:
# Adding colors to the dataframe for negative profit
movies["profit_colors"] = movies.profit.apply(lambda x: "red" if x < 0 else "limegreen")

In [ ]:
#Plot profit vs budget
# Plot a scatter or a joint plot between the columns budget and profit and write a few words on what you observed
movies.plot(kind = "scatter", x = "profit", y = "budget", c = "profit_colors", title = "Profit vs. Budget")
plt.show()

#### Observations

- We see couple of movies with large budgets having a negative value for Profit (ie., incurring a loss)
- There is one movie which has a high budget (approx. `250 Million`) and a very high profit (approx. `700 Million`)
- We cannot accurately judge the trend of movie profit based on budget looking at the scatter plot

Dropping the column `profit_colors` as it's no longer required

In [ ]:
movies.drop(columns = "profit_colors", inplace = True)

The dataset contains the 100 best performing movies from the year 2010 to 2016. However, the scatter plot tells a different story. You can notice that there are some movies with negative profit. Although good movies do incur losses, but there appear to be quite a few movie with losses. What can be the reason behind this? Lets have a closer look at this by finding the movies with negative profit.

Extract the movies with a negative profit and store them in a new dataframe - neg_profit

In [ ]:
#Find the movies with negative profit
neg_profit = movies[movies.profit < 0]

In [ ]:
# Verifying that only negative profit movies are present in the `neg_profit` dataframe
neg_profit.profit.value_counts()

In [ ]:
# To check if we have "Tangled" movie
neg_profit[neg_profit["Title"] == "Tangled"]

Can we spot the movie `Tangled` in the dataset? Although its one of the highest grossing movies of all time, it has negative profit as per this result. If we cross check the gross values of this movie (link: https://www.imdb.com/title/tt0398286/), we can see that the gross in the dataset accounts only for the domestic gross and not the worldwide gross. This is true for may other movies also in the list.

- ### Subtask 2.3: The General Audience and the Critics

We might have noticed the column `MetaCritic` in this dataset. This is a very popular website where an average score is determined through the scores given by the top-rated critics. Second, we also have another column `IMDb_rating` which tells us the IMDb rating of a movie. This rating is determined by taking the average of hundred-thousands of ratings from the general audience. 

As a part of this subtask, we find out the highest rated movies which have been liked by critics and audiences alike.
1. Firstly we will notice that the `MetaCritic` score is on a scale of `100` whereas the `IMDb_rating` is on a scale of 10. First convert the `MetaCritic` column to a scale of 10.
2. Now, to find out the movies which have been liked by both critics and audiences alike and also have a high rating overall, we need to -
    - Create a new column `Avg_rating` which will have the average of the `MetaCritic` and `Rating` columns
    - Retain only the movies in which the absolute difference(using abs() function) between the `IMDb_rating` and `Metacritic` columns is less than 0.5. Refer to this link to know how abs() funtion works - https://www.geeksforgeeks.org/abs-in-python/ .
    - Sort these values in a descending order of `Avg_rating` and retain only the movies with a rating equal to or greater than `8` and store these movies in a new dataframe `UniversalAcclaim`.
    

In [ ]:
# Change the scale of MetaCritic
movies["MetaCritic"] = movies["MetaCritic"] / 10

In [ ]:
# Checking distinct MetaCritic values to check if they are converted to a scale of 10
movies.MetaCritic.unique()

In [ ]:
# Find the average ratings
movies["Avg_rating"] = (movies["MetaCritic"] + movies["IMDb_rating"])/2

In [ ]:
# Find the movies with metacritic-Imdb rating < 0.5 and also with an average rating of >= 8 (sorted in descending order)
UniversalAcclaim = movies[ (abs(movies["MetaCritic"] - movies["IMDb_rating"]) < 0.5) & (movies["Avg_rating"] >= 8) ].sort_values(by = "Avg_rating", ascending = False).copy()

In [ ]:
UniversalAcclaim

- ### Subtask 2.4: Find the Most Popular Trios - I

If we are a producer looking to make a blockbuster movie, there will primarily be three lead roles in the movie and we wish to cast the most popular actors for it. Now, since we don't want to take a risk, you will cast a trio which has already acted in together in a movie before. The metric that we've chosen to check the popularity is the Facebook likes of each of these actors.

Our objective is to find the trios which has the most number of Facebook likes combined. That is, the sum of `actor_1_facebook_likes`, `actor_2_facebook_likes` and `actor_3_facebook_likes` should be maximum.
We find out the top 5 popular trios, and output their names in a list.


In [ ]:
# Grouping the actors based on their name and getting sum of their facebook likes. Once we sum up the values, we will
# get the total facebook likes of each trio of actors into a new dataframe "fb_actors_likes"
fb_actors_likes = pd.pivot_table(data = movies, index = ["actor_1_name","actor_2_name","actor_3_name"], values = ["actor_1_facebook_likes","actor_2_facebook_likes","actor_3_facebook_likes"], aggfunc = np.sum).sum(axis = 1)

In [ ]:
# Getting the 5 most popular trio of actors
fb_actors_likes.sort_values(ascending = False).head(5)

In [ ]:
# Getting the 5 most popular trio of actors into a list
popular_trios_5 = fb_actors_likes.sort_values(ascending = False).head(5).index.to_list()
popular_trios_5

- ### Subtask 2.5: Find the Most Popular Trios - II

In the previous subtask we found the popular trio based on the total number of facebook likes. Let's add a small condition to it and make sure that all three actors are popular. The condition is **none of the three actors' Facebook likes should be less than half of the other two**. For example, the following is a valid combo:
- actor_1_facebook_likes: 70000
- actor_2_facebook_likes: 40000
- actor_3_facebook_likes: 50000

But the below one is not:
- actor_1_facebook_likes: 70000
- actor_2_facebook_likes: 40000
- actor_3_facebook_likes: 30000

since in this case, `actor_3_facebook_likes` is 30000, which is less than half of `actor_1_facebook_likes`.

Having this condition ensures that we aren't getting any unpopular actor in your trio (since the total likes calculated in the previous question doesn't tell anything about the individual popularities of each actor in the trio.).

We can do a manual inspection of the top 5 popular trios you have found in the previous subtask and check how many of those trios satisfy this condition. Also, which is the most popular trio after applying the condition above?

### **Write your answers below.**

- **`No. of trios that satisfy the above condition:`** 3

- **`Most popular trio after applying the condition:`** 'Leonardo DiCaprio', 'Tom Hardy', 'Joseph Gordon-Levitt'

In [ ]:
# Creating a new dataframe that holds the trio of actors along with their facebook likes
fb_actors_likes_ind = pd.pivot_table(data = movies, index = ["actor_1_name","actor_2_name","actor_3_name"], values = ["actor_1_facebook_likes","actor_2_facebook_likes","actor_3_facebook_likes"], aggfunc = np.sum)

In [ ]:
# Iterating over each row and figure out the actors with facebook likes less than half of the other two
for lab,row in fb_actors_likes_ind.iterrows():
    if (row["actor_1_facebook_likes"]< row["actor_2_facebook_likes"]/2) | (row["actor_1_facebook_likes"]< row["actor_3_facebook_likes"]/2):
        fb_actors_likes_ind.loc[lab, "valid_like"] = "no"
    elif (row["actor_2_facebook_likes"]< row["actor_1_facebook_likes"]/2) | (row["actor_2_facebook_likes"]< row["actor_3_facebook_likes"]/2):
        fb_actors_likes_ind.loc[lab, "valid_like"] = "no"
    elif (row["actor_3_facebook_likes"]< row["actor_1_facebook_likes"]/2) | (row["actor_3_facebook_likes"]< row["actor_2_facebook_likes"]/2):
        fb_actors_likes_ind.loc[lab, "valid_like"] = "no"
    else:
        fb_actors_likes_ind.loc[lab, "valid_like"] = "yes"

In [ ]:
# Creating "total_likes" column to add the likes of all three actors
fb_actors_likes_ind["total_likes"] = fb_actors_likes_ind["actor_1_facebook_likes"] + fb_actors_likes_ind["actor_2_facebook_likes"] + fb_actors_likes_ind["actor_3_facebook_likes"]

In [ ]:
pop_trio_5pop = fb_actors_likes_ind.loc[fb_actors_likes_ind["valid_like"] != "no", :].sort_values(by = "total_likes", ascending = False).head(5).index.tolist()
pop_trio_5pop

- ### Subtask 2.6: Runtime Analysis

There is a column named `Runtime` in the dataframe which primarily shows the length of the movie. It might be interesting to see how this variable this distributed. We plot a `histogram` or `distplot` of seaborn to find the `Runtime` range most of the movies fall into.

In [ ]:
# Runtime histogram/density plot
movies.Runtime.plot(kind = "hist", title = "Runtime distribution", color = "green", alpha = 0.5, figsize = (10,5))
plt.show()

Most of the movies appear to be sharply 2 hour-long.

- ### Subtask 2.7: R-Rated Movies

Although R rated movies are restricted movies for the under 18 age group, still there are vote counts from that age group. Among all the R rated movies that have been voted by the under-18 age group, find the top 10 movies that have the highest number of votes i.e.`CVotesU18` from the `movies` dataframe. We store these in a dataframe named `PopularR`.

In [ ]:
# Write your code here
PopularR = movies[ movies["content_rating"] == "R" ].sort_values(by = "CVotesU18", ascending = False).head(10)

In [ ]:
PopularR.reset_index(drop = True)

These kids are watching `Deadpool` a lot.

## Task 3 : Demographic analysis

If we take a look at the last columns in the dataframe, most of these are related to demographics of the voters (in the last subtask, i.e., 2.8, you made use one of these columns - CVotesU18). We also have three genre columns indicating the genres of a particular movie. We will extensively use these columns for the third and the final stage of our assignment wherein we will analyse the voters across all demographics and also see how these vary across various genres. So without further ado, let's get started with `demographic analysis`.

-  ###  Subtask 3.1 Combine the Dataframe by Genres

There are 3 columns in the dataframe - `genre_1`, `genre_2`, and `genre_3`. As a part of this subtask, we need to aggregate a few values over these 3 columns. 
1. First,we create a new dataframe `df_by_genre` that contains `genre_1`, `genre_2`, and `genre_3` and all the columns related to **CVotes/Votes** from the `movies` data frame. There are 47 columns to be extracted in total.
2. Now, Add a column called `cnt` to the dataframe `df_by_genre` and initialize it to one.
3. First group the dataframe `df_by_genre` by `genre_1` and find the sum of all the numeric columns such as `cnt`, columns related to CVotes and Votes columns and store it in a dataframe `df_by_g1`.
4. Perform the same operation for `genre_2` and `genre_3` and store it dataframes `df_by_g2` and `df_by_g3` respectively. 
5. Now that you have 3 dataframes performed by grouping over `genre_1`, `genre_2`, and `genre_3` separately, it's time to combine them. For this, add the three dataframes and store it in a new dataframe `df_add`, so that the corresponding values of Votes/CVotes get added for each genre.
6. The column `cnt` on aggregation has basically kept the track of the number of occurences of each genre.Subset the genres that have atleast 10 movies into a new dataframe `genre_top10` based on the `cnt` column value.
7. Now, take the mean of all the numeric columns by dividing them with the column value `cnt` and store it back to the same dataframe. We will be using this dataframe for further analysis in this task unless it is explicitly mentioned to use the dataframe `movies`.
8. Since the number of votes can't be a fraction, type cast all the CVotes related columns to integers. Also, round off all the Votes related columns upto two digits after the decimal point.


In [ ]:
# Create the dataframe df_by_genre
# First create a new dataframe df_by_genre that contains genre_1, genre_2, and genre_3 and all the columns related to CVotes/Votes 
# from the movies data frame. There are 47 columns to be extracted in total
df_by_genre = movies.loc[:, "genre_1":"VotesnUS"]
df_by_genre.shape

In [ ]:
# Drop unnecessary columns "MetaCritic" and "Runtime"
df_by_genre.drop(columns = ["MetaCritic","Runtime"], inplace = True)

In [ ]:
# Checking the shape to check if we have the required 47 columns
df_by_genre.shape

In [ ]:
# Create a column cnt and initialize it to 1
df_by_genre["cnt"] = 1

In [ ]:
# Checking data of 'cnt' column
df_by_genre.cnt.value_counts()

In [ ]:
# Group the movies by individual genres
# First group the dataframe `df_by_genre` by `genre_1` and find the sum of all the numeric columns such as `cnt`, 
# columns related to CVotes and Votes columns and store it in a dataframe `df_by_g1`
# Perform the same operation for genre_2 and genre_3 and store it dataframes df_by_g2 and df_by_g3 respectively

df_by_g1 = df_by_genre.groupby(by = "genre_1").agg(np.sum)
df_by_g2 = df_by_genre.groupby(by = "genre_2").agg(np.sum)
df_by_g3 = df_by_genre.groupby(by = "genre_3").agg(np.sum)

In [ ]:
# All three genres have similar names and can be grouped together
print("genre_1", df_by_g1.index.unique)
print("genre_2", df_by_g2.index.unique)
print("genre_3", df_by_g3.index.unique)

In [ ]:
# Add the grouped data frames and store it in a new data frame
# Now that you have 3 dataframes performed by grouping over genre_1, genre_2, and genre_3 separately, it's time to combine them.
# For this, add the three dataframes and store it in a new dataframe df_add, so that the corresponding values of Votes/CVotes get added for each genre.
# There is a function called add() in pandas which lets you do this. You can refer to this link to see how this function works. 
# https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.add.html

df_add = df_by_g1.add(df_by_g2, fill_value = 0).add(df_by_g3, fill_value = 0)

In [ ]:
df_add.shape

In [ ]:
# Extract genres with atleast 10 occurences
# The column cnt on aggregation has basically kept the track of the number of occurences of each genre.
# Subset the genres that have atleast 10 movies into a new dataframe genre_top10 based on the cnt column value
genre_top10 = df_add[df_add["cnt"] >= 10]

In [ ]:
# Take the mean for every column by dividing with cnt 
# Now, take the mean of all the numeric columns by dividing them with column value cnt and store it back to the same dataframe.
# We will be using this dataframe for further analysis in this task unless it is explicitly mentioned to use the dataframe movies
for each_col in genre_top10.columns[:-1]:
    genre_top10[each_col] /= genre_top10["cnt"].copy()

Since the number of votes can't be a fraction, type cast all the CVotes related columns to integers. Also, round off all the Votes related columns upto two digits after the decimal point

In [ ]:
# Rounding off the columns of Votes to two decimals
genre_top10.loc[:,"VotesM":"VotesnUS"] = genre_top10.loc[:,"VotesM":"VotesnUS"].apply(lambda x: round(x,2))

In [ ]:
# Convert all CVotes columns to int
list_cvotes = genre_top10.loc[:,"CVotes10":"CVotesnUS"].columns
genre_top10[list_cvotes] = genre_top10[list_cvotes].astype(int)

In [ ]:
genre_top10.head()

If we take a look at the final dataframe that you have gotten, you will see that you now have the complete information about all the demographic (Votes- and CVotes-related) columns across the top 10 genres. We can use this dataset to extract exciting insights about the voters!

-  ###  Subtask 3.2: Genre Counts!

Now let's derive some insights from this data frame. Make a bar chart plotting different genres vs cnt using seaborn.

In [ ]:
# Countplot for genres
sns.barplot(data = genre_top10, x = genre_top10.index, y = "cnt")
plt.xticks(rotation = 90)
plt.show()

`Drama` is the tallest bar indicating that they are most produced.

-  ###  Subtask 3.3: Gender and Genre

If we have closely looked at the Votes- and CVotes-related columns, you might have noticed the suffixes `F` and `M` indicating Female and Male. Since we have the vote counts for both males and females, across various age groups, let's now see how the popularity of genres vary between the two genders in the dataframe. 

1. Make the first heatmap to see how the average number of votes of males is varying across the genres. Use seaborn heatmap for this analysis. The X-axis should contain the four age-groups for males, i.e., `CVotesU18M`,`CVotes1829M`, `CVotes3044M`, and `CVotes45AM`. The Y-axis will have the genres and the annotation in the heatmap tell the average number of votes for that age-male group. 

2. Make the second heatmap to see how the average number of votes of females is varying across the genres. Use seaborn heatmap for this analysis. The X-axis should contain the four age-groups for females, i.e., `CVotesU18F`,`CVotes1829F`, `CVotes3044F`, and `CVotes45AF`. The Y-axis will have the genres and the annotation in the heatmap tell the average number of votes for that age-female group. 

3. Make sure that we plot these heatmaps side by side using `subplots` so that you can easily compare the two genders and derive insights.

4.Repeat subtasks 1 to 3, but now instead of taking the CVotes-related columns, we need to do the same process for the Votes-related columns. These heatmaps will show you how the two genders have rated movies across various genres.

-  Note : Use `genre_top10` dataframe for this subtask

In [ ]:
# 1st set of heat maps for CVotes-related columns

male_cvotes = pd.pivot_table(data=genre_top10, values = ["CVotesU18M","CVotes1829M","CVotes3044M","CVotes45AM"], index = genre_top10.index, 
               aggfunc = np.mean)
female_cvotes = pd.pivot_table(data=genre_top10, values = ["CVotesU18F","CVotes1829F","CVotes3044F","CVotes45AF"], index = genre_top10.index, 
               aggfunc = np.mean)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize = [14,6])

sns.heatmap(male_cvotes, cmap = "RdYlGn", annot = True, ax = ax[0], fmt = "d")
sns.heatmap(female_cvotes, cmap = "RdYlGn", annot = True, ax = ax[1], fmt = "d")
plt.tight_layout()

plt.show()

**`Inferences:`** A few inferences that can be seen from the heatmap above is that males have voted more than females, and Sci-Fi appears to be most popular among the 18-29 age group irrespective of their gender. Additionally,
- Inference 1: We can see that all movie genres are least voted by Under 18 male and female group. This could be because there could be parental restrictions on the number aof movies that can be seen by these teenagers. There could also be restriction based on content rating which might affect their voting. We still have to verify this cause with other datasets
- Inference 2: `Animation` is the least voted genre by 45 & above female age group whereas it's `Romance` for 45 & above male age group
- Inference 3: `Sci-Fi` is the most voted genre by both male and female 30-44 age group

In [ ]:
# 2nd set of heat maps for Votes-related columns

male_votes = pd.pivot_table(data=genre_top10, values = ["VotesU18M","Votes1829M","Votes3044M","Votes45AM"], index = genre_top10.index, 
               aggfunc = np.mean)
female_votes = pd.pivot_table(data=genre_top10, values = ["VotesU18F","Votes1829F","Votes3044F","Votes45AF"], index = genre_top10.index, 
               aggfunc = np.mean)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize = [14,6])

sns.heatmap(male_votes, cmap = "RdYlGn", annot = True, ax = ax[0], fmt = "n")
sns.heatmap(female_votes, cmap = "RdYlGn", annot = True, ax = ax[1], fmt = "n")
plt.tight_layout()

plt.show()

**`Inferences:`** Sci-Fi appears to be the highest rated genre in the age group of U18 for both males and females. Also, females in this age group have rated it a bit higher than the males in the same age group. Additionally,
- Inference 1: *`Romance`* is the worst rated genre by both male and female with age 45 & above. In fact, as we can see from the pattern, the Avg rating decreases through age (U18, U1829, U3044, U45A) for both male and female age groups
- Inference 2: `Animation` movies are highest rated by 18-29 females while `Sci-Fi` are the highest rated by 18-29 males
- Inference 3: `Crime` movies are rated higher by U18 males compared to U18 females

-  ###  Subtask 3.4: US vs non-US Cross Analysis

The dataset contains both the US and non-US movies. Let's analyse how both the US and the non-US voters have responded to the US and the non-US movies.

1. Create a column `IFUS` in the dataframe `movies`. The column `IFUS` should contain the value "USA" if the `Country` of the movie is "USA". For all other countries other than the USA, `IFUS` should contain the value `non-USA`.


2. Now make a boxplot that shows how the number of votes from the US people i.e. `CVotesUS` is varying for the US and non-US movies. Make use of the column `IFUS` to make this plot. Similarly, make another subplot that shows how non US voters have voted for the US and non-US movies by plotting `CVotesnUS` for both the US and non-US movies. Write any of your two inferences/observations from these plots.


3. Again do a similar analysis but with the ratings. Make a boxplot that shows how the ratings from the US people i.e. `VotesUS` is varying for the US and non-US movies. Similarly, make another subplot that shows how `VotesnUS` is varying for the US and non-US movies. Write any of your two inferences/observations from these plots.

Note : Use `movies` dataframe for this subtask.

In [ ]:
# Creating IFUS column
movies["IFUS"] = movies.Country.apply(lambda x: "USA" if x == "USA" else "non-USA")

In [ ]:
# Setting plot style
plt.style.use("ggplot")

In [ ]:
# Box plot - 1: CVotesUS(y) vs IFUS(x)
fix, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (12,4))
sns.boxplot(data = movies, x = "IFUS", y = "CVotesUS", ax = ax[0])
sns.boxplot(data = movies, x = "IFUS", y = "CVotesnUS", ax = ax[1])

plt.tight_layout()
plt.show()

**`Inferences:`** Write your two inferences/observations below:
- Inference 1: People from USA tend to vote more for USA movies than non-USA movies
- Inference 2: People from non-USA have more range of votes above 300K for USA movies and there are even some values that might be considered outliers. In contrast, there is no outlier for non-USA movies and upper ceiling of votes is capped around ~300K

In [ ]:
# Box plot - 2: VotesUS(y) vs IFUS(x)
fix, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (12,4))
sns.boxplot(data = movies, x = "IFUS", y = "VotesUS", ax = ax[0])
sns.boxplot(data = movies, x = "IFUS", y = "VotesnUS", ax = ax[1])

plt.tight_layout()
plt.show()

**`Inferences:`**
- Inference 1: USA voters have not rated any non-USA movie above 8.2 whereas we have values over that range for USA movies
- Inference 2: non-USA voters have not rated a non-USA movie below 7.4 whereas they have rated some USA movies below that value

-  ###  Subtask 3.5:  Top 1000 Voters Vs Genres

We might have also observed the column `CVotes1000`. This column represents the top 1000 voters on IMDb and gives the count for the number of these voters who have voted for a particular movie. Let's see how these top 1000 voters have voted across the genres. 

1. Sort the dataframe genre_top10 based on the value of `CVotes1000`in a descending order.

2. Make a seaborn barplot for `genre` vs `CVotes1000`.

In [ ]:
# Sorting by CVotes1000
genre_top10.sort_values(by = "CVotes1000", ascending = False, inplace = True)

In [ ]:
# Bar plot
sns.barplot(data = genre_top10, x = genre_top10.index, y = "CVotes1000")
plt.xticks(rotation = 90)
plt.show()

**`Inferences:
- Inference 1: Sci-Fi is the most popular genre voted by Top 1000 IMDb voters
- Inference 2: Action is the second most popular genre voted by Top 1000 IMDb voters
- Inference 3: The genre `Romance` seems to be most unpopular among the top 1000 voters